In [3]:
from PIL import Image 
import glob
import numpy as np
import random

In [5]:
#making list to store img data and clearing them to not have residue
img_list=[]
img1_list=[]
img_list.clear()
img1_list.clear()

#making arrays to store notes and memes    
xm=np.empty((256,256,3))
xn=np.empty((256,256,3))

#importing memes images
for file in glob.glob('C:\\Users\\Gourav\\Desktop\\programs\\DL\\reshaped\\Memes\\*.jpg'):
    img = Image.open(file) 
    img_list.append(img)

In [6]:
#storing memes data as arrays
xm=np.array(img_list[0])
xm=xm.reshape((256*256*3,1))

for i in range(len(img_list)-1):
    temp=np.array(img_list[i+1])
    temp=temp.reshape((256*256*3,1))
    xm=np.append(xm,temp,axis=1)

In [7]:
#importing notes images
for file1 in glob.glob('C:\\Users\\Gourav\\Desktop\\programs\\DL\\reshaped\\Notes\\*.jpg'):
    img1 = Image.open(file1) 
    img1_list.append(img1)

#storing notes data as arrays
xn=np.array(img1_list[0])
xn=xn.reshape((256*256*3,1))
    
for i in range(len(img1_list)-1):
    temp1=np.array(img1_list[i+1])
    temp1=temp1.reshape((256*256*3,1))
    xn=np.append(xn,temp1,axis=1)

In [8]:
#making input vector by appending memes and notes array
x=np.append(xm,xn,axis=1)

In [9]:
#making labels for input
#0 means memes 1 means notes
y0=np.zeros((1,800))
y1=np.ones((1,800))
y=np.append(y0,y1)
Y=y.reshape(1,1600)
print(y.shape)

(1600,)


In [10]:
#shuffle data
permutation = list(np.random.permutation(1600))
x_arr=np.copy(x)
y_arr=np.copy(Y)
#shuffled_X = x_arr[:, permutation]
#shuffled_Y = y_arr[permutation]
#shuffled_Y.reshape(1,1600)
random.seed(1)
np.random.shuffle(np.transpose(x_arr))
np.transpose(x_arr)

random.seed(1)
shuff_y=np.random.shuffle(np.transpose(y_arr))
np.transpose(y_arr)

print(y_arr.shape)
print(x_arr.shape)

(1, 1600)
(196608, 1600)


In [15]:
#make test train validation
x_train=np.array(x_arr[:1500])
y_train=np.array(y_arr[:1500])

x_test=np.array(x_arr[1500:1550])
y_test=np.array(y_arr[1500:1550])

x_val=np.array(x_arr[1550:1600])
y_val=np.array(y_arr[1550:1600])

In [20]:
#make one hot
def one_hot(x):
    Y = np.zeros((2,int(x.shape)))
    for n,i in enumerated(range(len(x))):
        Y[int(x[i]),i]=1
    return Y

#Y_train=np.zeros_like(y_train)
#Y_test=np.zeros_like(y_test)
#Y_val=np.zeros_like(y_val)

print(type(y_train.shape))
Y_train=one_hot(y_train)
Y_test=one_hot(y_test)
Y_val=one_hot(y_val)

<class 'tuple'>


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'